In [1]:
import numpy as np
from lab1.plot import plotter
from lab2 import methods as lab2
import methods

In [2]:
np.random.seed(12)


def generate_points(n, dim, f):
    X = np.random.rand(n, dim)
    y = []
    X_err = X + 0.2 * np.random.randn(n, dim)
    for x in X_err:
        y.append(f(x))
    return X, np.asarray(y)


def mse(X, y, w):
    res = 0
    for i in range(0, len(y)):
        x = 0
        for j in range(0, len(X[i])):
            x += X[i][j] * w[j]
        res += np.square(y[i] - x)
    return res / len(y)


def mse_func(X, y):
    return lambda w: mse(X, y, w)


def grad_calculator(x, func, n):
    h = 1e-5
    res = []
    for i in range(n):
        delta = np.zeros(n)
        delta[i] = h
        res.append((func(x + delta) - func(x - delta)) / (2 * h))
    return np.asarray(res)


def grad_func(f, n):
    return lambda x: grad_calculator(x, f, n)

In [3]:
def target_func(x):
    return 5 * x[0] + 2 * x[1]

def gauss_func(p):
    return lambda x: p[0] * x[0] + p[1] * x[1]

n = 1000
dim = 2
(X, y) = generate_points(n, dim, target_func)
f = mse_func(X, y)
grad = grad_func(f, dim)

In [4]:
def print_info(name, start, points, grad_calc, func_calc):
    print("start:", start)
    print("{}:".format(name))
    print(points[-1], f(points[-1]))
    print("gradient calculations:", grad_calc)
    print("function calculations:", func_calc)


def plot_from_start(start):
    lr = lambda x: 2

   # (points1, grad_calc1, func_calc1) = lab2.sgd_with_momentum(f, grad, start, learning_rate=lr)
   # print_info("Momentum", start, points1, grad_calc1, func_calc1)

   # (points2, grad_calc2, func_calc2) = lab2.sgd_nesterov(f, grad, start, learning_rate=lr)
   # print_info("Nesterov", start, points2, grad_calc2, func_calc2)

    #(points3, grad_calc3, func_calc3) = lab2.sgd_adagrad(f, grad, start, learning_rate=lr)
    #print_info("AdaGrad", start, points3, grad_calc3, func_calc3)

    #(points4, grad_calc4, func_calc4) = lab2.sgd_rmsprop(f, grad, start, learning_rate=lr)
    #print_info("RMSProp", start, points4, grad_calc4, func_calc4)

    #(points5, grad_calc5, func_calc5) = lab2.sgd_adam(f, grad, start, learning_rate=lr)
    #print_info("Adam", start, points5, grad_calc5, func_calc5)

    #(points6, grad_calc6, func_calc6) = methods.bfgs(f, grad, start)
    #print_info("BFGS", start, points6, grad_calc6, func_calc6)

    #(points7, grad_calc7, func_calc7) = methods.l_bfgs(f, grad, start)
   # print_info("L-BFGS", start, points7, grad_calc7, func_calc7)

    (points8, grad_calc8, func_calc8) = methods.gauss_newton(gauss_func, methods.jacobian, X, y, start)
    print_info("Gauss-Newton", start, points8, grad_calc8, func_calc8)

    #plotter.multiple_points_over_contour(f, points1, points2, name1="Momentum", name2="Nesterov").show()

   # plotter.multiple_points_over_contour(f, points4, points3, name1="RMSProp", name2="AdaGrad",).show()

   # plotter.multiple_points_over_contour(f, points5, name1="Adam").show()

   # plotter.multiple_points_over_contour(f, points6, name1="BFGS").show()

   # plotter.multiple_points_over_contour(f, points7, name1="L-BFGS").show()

    plotter.multiple_points_over_contour(f, points8, name1="Gauss-Newton").show()

In [5]:
plot_from_start([0, 0])

len(x): 1000


IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
plot_from_start([-1, 2])

In [ ]:
plot_from_start([-0.5, -1])

In [ ]:
plot_from_start([-0.5, 0])

In [ ]:
plot_from_start([2, 3])

In [ ]:
plot_from_start([-10, 0])

In [ ]:
plot_from_start([0.9504983984550248, -0.766835117539493])

In [ ]:
#generate 4 dots
dots = plotter.generate_n_dots(4)

In [ ]:
for i in dots:
    plot_from_start(i)
